In [263]:
# 1. import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import time

In [290]:
def get_city_population(city):
    url = ("https://en.wikipedia.org/wiki/"+city)
    response = requests.get(url)
    time.sleep(2)   #quick solution
    soup = BeautifulSoup(response.content, "html.parser")
    table_string = soup.find(text=re.compile('^Population\Z'))
    parent = table_string.find_parent('tr')
    description = []
    population = []
    population_string = parent.get_text("|",strip='True')
    description.append(population_string.split('|')[0])
    population_string2 = population_string.split('|')[-1]
    if(population_string2.startswith('[')):
        population.append('')
    else:
        population.append(population_string2)

    siblings = parent.next_siblings
    for sib in siblings:
        if(sib['class'] != ['mergedrow']):
            break
        for ch in sib.children:
            if(ch['class']==['infobox-label']):
                temp = ch.get_text(strip='True').replace('\xa0',' ')
                temp = re.sub(r'[^a-zA-Z ]','',temp)
                description.append(temp)
            elif(ch['class']==['infobox-data']):
                population.append(ch.get_text(strip='True').replace('\xa0',' '))
    population_table = pd.DataFrame({"Description":description,
                       "Population":population
                      })
    return population_table

In [291]:
print(get_city_population('New_York_City'))

  Description                             Population
0  Population                                       
1       Total                              8,804,190
2        Rank  1stin the United States1stin New York
3     Density        29,302.37/sq mi (11,313.68/km2)
4       Metro                       20,140,470 (1st)
